In [22]:
from infisical import InfisicalClient
import os
from sentence_transformers import SentenceTransformer

from openai import OpenAI
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, DeclarativeBase, mapped_column
from pgvector.sqlalchemy import Vector

import json

if_client = InfisicalClient(token = os.environ.get('INFISICAL_TOKEN'))

DWH_USER   = if_client.get_secret('DWH_PG_USER')
DWH_PW     = if_client.get_secret('DWH_PG_PW')
DWH_HOST   = if_client.get_secret('DWH_PG_HOST')
DWH_DBNAME = if_client.get_secret('DWH_PG_DBNAME')

postgres_conn = f'postgresql+psycopg2://{DWH_USER.secret_value}:{DWH_PW.secret_value}@{DWH_HOST.secret_value}:5432/{DWH_DBNAME.secret_value}'

model = SentenceTransformer('BAAI/bge-small-en-v1.5')

os.environ['OPENAI_API_KEY'] = if_client.get_secret('OPENAI_API_KEY').secret_value
openai_client = OpenAI()

In [17]:
def llm(prompt, model = 'gpt-3.5-turbo-1106', client = openai_client):
    return client.chat.completions.create(
        model = model,
        messages = prompt,
        stream = True,
    )

In [7]:
class Base(DeclarativeBase):
    pass

class SparkEmbeddings(Base):
    __tablename__ = "spark_embeddings"
    spark_id = mapped_column(sa.VARCHAR(36), primary_key = True)
    map_id = mapped_column(sa.VARCHAR(36))
    entity_updated = mapped_column(sa.DateTime)
    title = mapped_column(sa.String)
    fulltext = mapped_column(sa.String)
    embedding = mapped_column(Vector(384))

In [10]:
query = 'Why is the sky blue?'
q_embedding = model.encode(query)

engine = create_engine(postgres_conn, echo = True, echo_pool = True)

# create session and add objects
with Session(engine) as session:
    query_results = session.query(SparkEmbeddings).\
        order_by(SparkEmbeddings.embedding.l2_distance(q_embedding)).\
        limit(5).all()

# print(type(query_results))
for r in query_results:
    print(r.title)

2023-11-22 15:11:49,396 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-22 15:11:49,397 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-22 15:11:49,505 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-22 15:11:49,507 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-22 15:11:49,613 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-22 15:11:49,614 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-22 15:11:49,728 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-22 15:11:49,738 INFO sqlalchemy.engine.Engine SELECT spark_embeddings.spark_id AS spark_embeddings_spark_id, spark_embeddings.map_id AS spark_embeddings_map_id, spark_embeddings.entity_updated AS spark_embeddings_entity_updated, spark_embeddings.title AS spark_embeddings_title, spark_embeddings.fulltext AS spark_embeddings_fulltext, spark_embeddings.embedding AS spark_embeddings_embedding 
FROM spark_embeddings ORDER BY spark_embeddings.embedding <-> %(embedding_1)s 
 

In [23]:
qdocs = json.dumps([{
    'source': f'<spark_id="r.spark_id"/>',
    'title': r.title,
    'content': r.fulltext,
} for r in query_results])

In [24]:
prompt = [
    {"role": "system", "content": "You're an honest, helpful and polite assistent answering questions from the provided context and only from context. If the context is not enough, you honestly admit that there is not enough data to answer the question."},
    {"role": "system", "content": "The context is provided as JSON array of objects with title, content and source keys, where content is the text and source is the source of the text. For each and every generated sentence you must provide the source of the sentence."},
    {"role": "system", "content": "Context: " + qdocs},
    {"role": "user", "content": query}
]

In [26]:
stream = llm(prompt)
for part in stream:
    print(part.choices[0].delta.content or "", end = "")

I'm afraid the provided context does not contain relevant information to answer that question.